In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_big_money = pd.read_csv('Big_Black_Money.csv')
df_big_money.info()

#Convert Date of Transaction to Date time data type

pd.to_datetime(df_big_money['Date of Transaction'])

#1. What are the Top 5 countries with the highest Total Transaction amount?

country_totals = df_big_money.groupby('Country')['Amount (USD)'].sum().sort_values(ascending=False)
top_5_countries = country_totals.head(5)
print(top_5_countries)

country_totals = df_big_money.groupby('Country')['Amount (USD)'].sum().sort_values(ascending=False)
country_totals

#2. What is the correlation between the Money Laundering Risk Score and the number of Shell Companies involved?

avg_shell_companies = df_big_money.groupby('Money Laundering Risk Score')['Shell Companies Involved'].mean().reset_index()
avg_shell_companies

#Visualization: lineplot
sns.lineplot(x='Money Laundering Risk Score', y='Shell Companies Involved', data=avg_shell_companies, marker='o', color = '#097969')

#Visualization: ScatteredPlot
sns.relplot(x='Money Laundering Risk Score', y='Shell Companies Involved', data=avg_shell_companies, marker='o', color = '#097969')

#3. TOP 10 industries in High-Risk Transactions(Risk Score > 7)

#Filter for High-Risks

high_risk_df = df_big_money[df_big_money['Money Laundering Risk Score'] > 7]
high_risk_df

# Count the frequency of each industry in high-risk transactions

industry_counts = high_risk_df['Industry'].value_counts()
industry_counts.head(10)

# Calculate the percentage of high-risk transactions for each industry

total_high_risk = len(high_risk_df)
industry_percentages = (industry_counts / total_high_risk) * 100

print("Top 10 Industries Most Frequently Associated with High-Risk Transactions:")
print(industry_percentages.head(10))

# Create a bar plot of the top 10 industries

plt.figure(figsize=(12, 6))
industry_percentages.head(10).plot(kind='bar', color = '#097969')
plt.title('Top 10 Industries in High-Risk Transactions (Risk Score > 7)')
plt.xlabel('Industry')
plt.ylabel('Percentage of High-Risk Transactions')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#4. What percentage of transactions involve Tax Haven Countries?

# List of common tax haven countries
tax_havens = [
    'Bermuda', 'Cayman Islands', 'British Virgin Islands', 'Luxembourg', 'Switzerland',
    'Ireland', 'Netherlands', 'Cyprus', 'Malta', 'Mauritius', 'Singapore', 'Hong Kong',
    'Panama', 'Bahamas', 'Jersey', 'Guernsey', 'Isle of Man', 'Liechtenstein', 'Monaco',
    'Andorra', 'Belize', 'Seychelles', 'Delaware', 'Nevada', 'Wyoming'
]

# Create a boolean column for tax haven involvement
df_big_money['Involves_Tax_Haven'] = df_big_money['Country'].isin(tax_havens)

# Calculate the percentage of transactions involving tax havens
total_transactions = len(df_big_money)
tax_haven_transactions = df_big_money['Involves_Tax_Haven'].sum()
percentage_tax_haven = (tax_haven_transactions / total_transactions) * 100

print(f"Total number of transactions: {total_transactions}")
print(f"Number of transactions involving tax havens: {tax_haven_transactions}")
print(f"Percentage of transactions involving tax havens: {percentage_tax_haven:.2f}%")

# Create a pie chart
plt.figure(figsize=(10, 6))
plt.pie([percentage_tax_haven, 100 - percentage_tax_haven], 
        labels=['Tax Haven', 'Non-Tax Haven'],
        autopct='%1.1f%%',
        startangle=90,
        colors=['#afe1af', '#097969'])
plt.title('Percentage of Transactions Involving Tax Haven Countries')
plt.axis('equal')
plt.show()

# Analyze the distribution of risk scores for tax haven vs non-tax haven transactions

tax_haven_risk = df_big_money[df_big_money['Involves_Tax_Haven']]['Money Laundering Risk Score']
non_tax_haven_risk = df_big_money[~df_big_money['Involves_Tax_Haven']]['Money Laundering Risk Score']

plt.figure(figsize=(10, 6))
plt.hist([tax_haven_risk, non_tax_haven_risk], color = ['#097969', '#afe1af'], label=['Tax Haven', 'Non-Tax Haven'], bins=15, alpha=0.7,)
plt.xlabel('Money Laundering Risk Score')
plt.ylabel('Number of Transactions')
plt.title('Distribution of Risk Scores: Tax Haven vs Non-Tax Haven Transactions')
plt.legend()
plt.show()

# Calculate average risk scores
avg_risk_tax_haven = tax_haven_risk.mean()
avg_risk_non_tax_haven = non_tax_haven_risk.mean()

print(f"\
Average Risk Score for Tax Haven Transactions: {avg_risk_tax_haven:.2f}")
print(f"Average Risk Score for Non-Tax Haven Transactions: {avg_risk_non_tax_haven:.2f}")

# Top 5 tax havens by transaction count
top_tax_havens = df_big_money[df_big_money['Involves_Tax_Haven']]['Country'].value_counts().head()
print("\
Top 5 Tax Havens by Transaction Count:")
print(top_tax_havens)

# Percentage of high-risk transactions (score > 7) involving tax havens
high_risk_tax_haven = df_big_money[(df_big_money['Involves_Tax_Haven']) & (df_big_money['Money Laundering Risk Score'] > 7)].shape[0]
high_risk_total = df_big_money[df_big_money['Money Laundering Risk Score'] > 7].shape[0]
percentage_high_risk_tax_haven = (high_risk_tax_haven / high_risk_total) * 100

print(f"\
Percentage of high-risk transactions (score > 7) involving tax havens: {percentage_high_risk_tax_haven:.2f}%")

#5. What financial institutions are handling the highest volume of high risk transactions?

# Filter for high-risk transactions (risk score > 7)
high_risk_transactions = df_big_money[df_big_money['Money Laundering Risk Score'] > 7]

# Group by Financial Institution and sum the transaction amounts
high_risk_by_institution = high_risk_transactions.groupby('Financial Institution')['Amount (USD)'].sum().sort_values(ascending=False)

# Display the top 5 financial institutions handling the highest volume of high-risk transactions
print("Top 5 Financial Institutions by High-Risk Transaction Volume:")
print(high_risk_by_institution.head())

# Plot the top 5 financial institutions
plt.figure(figsize=(10, 6))
high_risk_by_institution.head().plot(kind='bar', color='#097969')
plt.title('Top 5 Financial Institutions by High-Risk Transaction Volume')
plt.xlabel('Financial Institution')
plt.ylabel('Total Transaction Volume (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#6. What is the relationship between the Source Country and Destination Country for high value transaction?

# Define high-value transactions (e.g., top 10% by amount)
high_value_threshold = df_big_money['Amount (USD)'].quantile(0.9)
high_value_transactions = df_big_money[df_big_money['Amount (USD)'] >= high_value_threshold]

# Create country pairs and count their occurrences
country_pairs = high_value_transactions.groupby(['Country', 'Destination Country']).size().reset_index(name='Count')
country_pairs = country_pairs.sort_values('Count', ascending=False).head(10)

print("Top 10 Source-Destination Country Pairs for High-Value Transactions:")
print(country_pairs)

# Visualize the top country pairs
plt.figure(figsize=(12, 6))
sns.barplot(x='Count', y='Country', hue='Destination Country', data=country_pairs)
plt.title('Top 10 Source-Destination Country Pairs for High-Value Transactions')
plt.xlabel('Number of Transactions')
plt.ylabel('Source Country')
plt.legend(title='Destination Country', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Calculate the percentage of domestic high-value transactions
domestic_transactions = high_value_transactions[high_value_transactions['Country'] == high_value_transactions['Destination Country']]
domestic_percentage = (len(domestic_transactions) / len(high_value_transactions)) * 100

print(f"\
Percentage of domestic high-value transactions: {domestic_percentage:.2f}%")

# Top 5 countries involved in high-value transactions (either as source or destination)
top_countries = pd.concat([high_value_transactions['Country'], high_value_transactions['Destination Country']]).value_counts().head()
print("\
Top 5 countries involved in high-value transactions:")
print(top_countries)
print('This analysis reveals interesting patterns in high-value transactions between countries.' 
     'Russia, UK, China, South Africa, and Switzerland are the most involved countries.' 
      'Only 10.80% of high-value transactions are domestic.')

#7. Look out for a trend of transaction amounts or risk scores over time?

# Convert 'Date of Transaction' to datetime
df_big_money['Date of Transaction'] = pd.to_datetime(df_big_money['Date of Transaction'])

# Set the date as the index
df_big_money.set_index('Date of Transaction', inplace=True)

# Resample data monthly and calculate mean amount and risk score
monthly_data = df_big_money.resample('M').agg({
    'Amount (USD)': 'mean',
    'Money Laundering Risk Score': 'mean'
})

# Plot the trends
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Transaction Amount Trend
ax1.plot(monthly_data.index, monthly_data['Amount (USD)'], color = '#097969')
ax1.set_title('Average Transaction Amount Over Time')
ax1.set_ylabel('Amount (USD)')

# Risk Score Trend
ax2.plot(monthly_data.index, monthly_data['Money Laundering Risk Score'], color = '#097969')
ax2.set_title('Average Money Laundering Risk Score Over Time')
ax2.set_ylabel('Risk Score')

plt.tight_layout()
plt.show()

# Print summary statistics
print(monthly_data.describe())

# Calculate correlation between amount and risk score
correlation = monthly_data['Amount (USD)'].corr(monthly_data['Money Laundering Risk Score'])
print(f"\
Correlation between Amount and Risk Score: {correlation:.2f}")

print('Correlation between Amount and Risk Score: -0.13', 
      "The correlation between amount and risk score is slightly negative," 
      "suggesting that higher transaction amounts don't necessarily correspond to higher risk scores.")
